# Exercise 5 - Automatically find Exchange links for DWM Query

In this demo we will see how to combine DWM and Iknaio to automatically find connections to exchanges given a bunch of crypto addresses mentioned some darkweb context

## Preparations

First, we install the graphsense-python package and define an API-key. An API-key for the [GraphSense](https://graphsense.github.io/) instance hosted by [Iknaio](https://www.ikna.io/) can be requested by sending an email to [contact@iknaio.com](contact@iknaio.com).

In [2]:
!pip install graphsense-python seaborn

import graphsense
from graphsense.api import bulk_api, general_api

import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

  Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 38.4 MB/s eta 0:00:00m eta 0:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 38.8 MB/s eta 0:00:00
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.0/325.0 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 52.5 MB/s eta 0:00:00 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 57.8 MB/s eta 0:00:0031m64.8 MB/s eta 0:00:01

[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
configuration = graphsense.Configuration(
    host = "https://api.ikna.io/",
    api_key = {
        'api_key': ""
    }
)

GraphSense supports Bitcoin-like UTXO and Ethereum-like Account-Model ledgers. Iknaio currently hosts BTC, LTC, BCH, ZEC, and ETH.

We are investigating Bitcoin transactions, therefore we set the default currency to Bitcoin **BTC**.

In [3]:
CURRENCY = 'btc'

We can test whether or client works, by checking what data the GraphSense endpoint provides

In [4]:
with graphsense.ApiClient(configuration) as api_client:
    api_instance = general_api.GeneralApi(api_client)
    api_response = api_instance.get_statistics()
    display(api_response)

{'currencies': [{'name': 'btc',
                 'no_address_relations': 7984317882,
                 'no_addresses': 1117419892,
                 'no_blocks': 785517,
                 'no_entities': 506028852,
                 'no_labels': 11478,
                 'no_tagged_addresses': 269790236,
                 'no_txs': 824257965,
                 'timestamp': 1681552500},
                {'name': 'bch',
                 'no_address_relations': 2425295666,
                 'no_addresses': 336921338,
                 'no_blocks': 788521,
                 'no_entities': 151942958,
                 'no_labels': 143,
                 'no_tagged_addresses': 15017913,
                 'no_txs': 366583821,
                 'timestamp': 1681595919},
                {'name': 'ltc',
                 'no_address_relations': 1724488590,
                 'no_addresses': 182641493,
                 'no_blocks': 2457244,
                 'no_entities': 78313960,
                 'no_labels': 181,

# 1. Load Starting Addresses from DWM


In [6]:
addresses = pd.read_csv(
    'https://raw.githubusercontent.com/iknaio/iknaio-api-tutorial/main/data/sextortion_addresses.csv',
    header=None,
    names=["address"]
)
addresses

,address
0,1JwRp2J8bQcoG8XTUbxQZaEj9QB4RB6zEa
1,1EZS92K4xJbymDLwG4F7PNF5idPE62e9XY
2,16B4HuSAJ4WRdCq7dzA5b4ASh6QQ7ytZWB
3,1EdX5vtBiHGmkqbJc7VRSuVMx9Kpgh53Tp
4,3Ch7RPfwkJ3wHhiBfA4CNc8SagGdjbZwVs
...,...
240,1BC1pvPUQF9QHg73ha4AEAhaoEvg6HmTbS
241,13QKq8RsvbJnLRbi5ZcVX1ziYW83tqvp1q
242,1DiEqE5R1Ktu7QCLUuJN31PNtpoBU41x2E
243,1NWybUp8ZJXKyDg2DR5MaePspforMPYbM3


# Q1. How many of the addresses are used?

Instead of querying each address individually, we just pass the dataframe of known sextortion payment addresses.

In [29]:
with graphsense.ApiClient(configuration) as clnt:
    blkapi = bulk_api.BulkApi(clnt)

    # documentation about available bulk operations can be found
    # here https://api.ikna.io/#/bulk/bulk_csv
    rcsv = blkapi.bulk_csv(
                CURRENCY,
                operation="get_address",
                body={
                    'address': addressDF['address'].to_list()
                },
                num_pages=1,
                _preload_content=False
              )
    respAddrDF = pd.read_csv(rcsv)
respAddrDF[["address", "balance_eur", "total_received_eur", "total_spent_eur", "in_degree", "out_degree", "no_incoming_txs", "no_outgoing_txs", "first_tx_height", "last_tx_height"]]

,address,balance_eur,total_received_eur,total_spent_eur,in_degree,out_degree,no_incoming_txs,no_outgoing_txs,first_tx_height,last_tx_height
0,1ZWFbUTUEQw7VMCzWu1SzfPm9HaiqP6rX,0.0,1534.20,1161.10,3,2,3,2,548067,560647
1,1e8o68StxCFLr6wdwKBrBqMQZc1VbFVMk,0.0,909.67,909.67,3,2,3,2,555912,557770
2,139XY4ZjWYqHMJvGCySuzXq7o6tGccKKrJ,0.0,957.29,973.32,7,2,7,1,543153,544107
3,3P5yeiyWLciKi28yY22LdRntJucSuedTq4,0.0,1295.29,1301.35,3,4,3,3,548894,552405
4,1KzMDhZLokkNd1kcxs2mgwXm97pVvnfRBC,0.0,1397.69,1370.09,2,2,2,2,552874,553642
...,...,...,...,...,...,...,...,...,...,...
240,1Lmk4eUXcmtVU6YQvaPJ4yihu4fEcKtkby,0.0,885.24,843.88,2,1,2,1,553043,553642
241,19EDGUoy7F1z4QjTpr67E7b6gn4uvefYJL,0.0,1086.50,1096.33,3,3,3,3,543183,544524
242,1L47wHe7FXWQ6pfPTbnykdX44FxQGstFeS,0.0,1237.92,1237.92,3,2,2,2,563885,566916
243,18YDAf11psBJSavARQCwysE7E89zSEMfGG,0.0,3641.37,3496.77,7,3,5,2,551191,552832


In [14]:
print(f"{len(respAddrDF)} addresses received {sum(respAddrDF['total_received_usd']):.2f} USD")

245 addresses received 886352.11 USD


# Q2: Are there direct links to exchanges?

They map to 96 distinct entities


# Q3: Can i link more addresses to our seed addresses?

We now fetch summary statistics for each entity.

In [19]:
with graphsense.ApiClient(configuration) as clnt:
  blkapi = bulk_api.BulkApi(api_client)
  rcsv = blkapi.bulk_csv(
                                 CURRENCY,
                                 operation = "get_entity",
                                 body={
                                     'entity': respAddrDF['entity'].drop_duplicates().to_list()
                                     },
                                 num_pages=1,
                                 _preload_content=False
                                 )
  respEntityDF = pd.read_csv(rcsv)

respEntityDF[
    ["best_address_tag_label",
     "root_address",
     "no_addresses",
     "balance_eur",
     "total_received_eur",
     "total_spent_eur",
     "in_degree",
     "out_degree",
     "no_incoming_txs",
     "no_outgoing_txs",
     "first_tx_height",
     "last_tx_height"]
     ]

,best_address_tag_label,root_address,no_addresses,balance_eur,total_received_eur,total_spent_eur,in_degree,out_degree,no_incoming_txs,no_outgoing_txs,first_tx_height,last_tx_height
0,sextortion spam,192CdbpYmpQhbpSZy5J9qyNE3YCxPpxdxv,1,0.0,0.86,0.87,1,2,1,1,543949,544640
1,sextortion spam,18eBGkYam1wjz1S77jz3VmADuYYFzhA3vB,1,0.0,7413.08,7847.13,17,1,19,1,556368,557342
2,sextortion spam,18vVjgfyC6sbLEEJvYHTXoWeuupZDtqyNs,1,0.0,807.16,881.37,1,2,1,1,557944,564024
3,sextortion spam,1FCGhC7ncVgfepxzaAzq8Qdq2ypjvfYHhF,1,0.0,661.63,661.63,1,2,1,1,562961,563038
4,sextortion spam,17C34w7vtuj8fG8CZLDeLLa3AYJFEWM8Li,1,0.0,366.96,366.96,1,1,1,1,542383,542431
...,...,...,...,...,...,...,...,...,...,...,...,...
91,sextortion spam,1Pdf1QMXH7e9957vhMskAFKQNi79eoa9Rm,2,0.0,111.78,111.78,3,4,3,2,557855,558468
92,sextortion spam,16LBDius3vg6ufFvnc7PGXfiTZgphuZgr5,2,0.0,3891.91,4079.90,9,3,9,3,555787,559368
93,sextortion spam,17iRfpgSwmJ6nLXR8evx6pUBo3R33S5LXB,4,0.0,5954.55,6086.09,14,3,14,2,537047,547878
94,sextortion spam,1NxiHuaorQsLsnNWqgo3iitVJeT2fMrtd7,16,0.0,13183.41,16594.01,15,1,18,1,547674,576358
